# Baseline
* Learns a user vector $u$ and an item vector $a$ and predicts $u_i + a_j$
* Minimizes $L = w_{ij} * (r_{ij} - u_i - a_j) ^ 2 + λ_u * (u_i - μ_u) ^ 2  + λ_a * (a_j - μ_a) ^ 2 $
* This can be computed efficiently via Alternating Least Squares

In [ ]:
medium = "anime"
dataset = "streaming"

In [ ]:
const version = "v1";

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../TrainingAlpha.ipynb");

In [ ]:
include("get_user_biases.jl");

In [ ]:
import NNlib: sigmoid
import ProgressMeter: @showprogress
import Statistics: mean

# Alternating Least Squares

In [ ]:
@memoize function get_user_partition(users, threadid, num_threads)
    [i for i in 1:length(users) if (users[i] % num_threads) + 1 == threadid]
end

function update_users!(users, items, ratings, weights, u, a, μ_uλ_u, Ω)
    Threads.@threads for i = 1:length(u)
        @inbounds u[i] = μ_uλ_u
    end
    T = Threads.nthreads()
    @sync for t = 1:T
        Threads.@spawn begin
            @inbounds for row in get_user_partition(users, t, T)
                i = users[row]
                j = items[row]
                r = ratings[row]
                w = weights[row]
                u[i] += (r - a[j]) * w
            end
        end
    end
    Threads.@threads for i = 1:length(u)
        @inbounds u[i] /= Ω[i]
    end
end;

In [ ]:
@memoize function get_counts(df, col)
    data = getfield(df, col)
    counts = StatsBase.countmap(data)
    Int32[counts[x] for x in data]
end

function get_weights(df, λ_wu, λ_wa, λ_wt)
    users = get_counts(df, :userid)
    items = get_counts(df, :itemid)
    w = Vector{typeof(λ_wt)}(undef, length(users))
    Threads.@threads for i = 1:length(w)
        w[i] = (users[i]^λ_wu) * (items[i]^λ_wa) * (λ_wt^(1 - df.updated_at[i]))
    end
    w
end;

function get_denom(weights, λ, users, num_users)
    Ω_u = Vector{eltype(weights)}(undef, num_users)
    Threads.@threads for i = 1:length(Ω_u)
        Ω_u[i] = λ
    end
    T = Threads.nthreads()
    @sync for t = 1:T
        Threads.@spawn begin
            @inbounds for row in get_user_partition(users, t, T)
                Ω_u[users[row]] += weights[row]
            end
        end
    end
    Ω_u
end

function train_model(λ, training, medium)
    μ_a, λ_u, λ_a, λ_wu, λ_wa, λ_wt = λ
    λ_u, λ_a = exp.((λ_u, λ_a))
    λ_wt = sigmoid(λ_wt)
    users, items, ratings = training.userid, training.itemid, training.rating
    weights = get_weights(training, λ_wu, λ_wa, λ_wt)

    u = zeros(typeof(λ_u), maximum(users))
    a = zeros(typeof(λ_a), num_items(medium))
    Ω_u = get_denom(weights, λ_u, users, length(u))
    Ω_a = get_denom(weights, λ_a, items, length(a))

    max_iters = 8
    @showprogress for _ = 1:max_iters
        update_users!(items, users, ratings, weights, a, u, μ_a * λ_a, Ω_a)
        update_users!(users, items, ratings, weights, u, a, 0, Ω_u)
    end
    u, a
end;

# Optimize Hyperparameters

In [ ]:
function make_prediction(users, items, u, a)
    r = Array{eltype(u)}(undef, length(users))
    Threads.@threads for i = 1:length(r)
        @inbounds r[i] = u[users[i]] + a[items[i]]
    end
    r
end;

function mse_and_beta(λ, training, test_input, test_output, medium)
    _, a = train_model(λ, training, medium)
    u = get_user_biases(test_input, λ, a, get_counts(training, :itemid))
    x = Array{eltype(a)}(undef, length(test_output.userid))
    Threads.@threads for i = 1:length(x)
        @inbounds x[i] = get(u, test_output.userid[i], 0) + a[test_output.itemid[i]]
    end
    y = test_output.rating
    w = [1 / c for c in get_counts(test_output, :userid)]
    xw = (x .* sqrt.(w))
    yw = (y .* sqrt.(w))
    β = (xw'xw + 1f-9) \ xw'yw
    L = loss(x * β, y, w, "rating")
    L, β
end;

In [ ]:
function average_item_rating(df)
    s = Dict()
    w = Dict()
    for (a, r) in zip(df.itemid, df.rating)
        if a ∉ keys(w)
            s[a] = 0
            w[a] = 0
        end
        s[a] += r
        w[a] += 1
    end
    mean([s[a] / w[a] for a in keys(w)])
end;

In [ ]:
function training_test_split(df::RatingsDataset, test_frac::Float64)
    userids = Random.shuffle(sort(collect(Set(df.userid))))
    n_train = Int(round(length(userids) * (1 - test_frac)))
    train_userids = Set(userids[1:n_train])
    test_userids = Set(userids[n_train+1:end])
    train_df = subset(df, df.userid .∈ (train_userids,))
    test_df = subset(df, df.userid .∈ (test_userids,))
    train_df, test_df
end;

In [ ]:
function build_model(medium, dataset, version; max_output_days, max_output_items)
    seed_rng!("TrainingAlphas/Baseline/Train")
    training = as_metric(
        get_split(
            dataset,
            "train",
            medium,
            [:userid, :itemid, :medium, :rating, :updated_at, :update_order],
        ),
        "rating",
    )
    training, test = training_test_split(training, 0.1)
    test_input, test_output = input_output_split(
        test,
        get_timestamp(dataset, :max_ts) - get_timestamp(Dates.Day(max_output_days)),
        max_output_items,
        true,
    )
    test = nothing

    if dataset == "training"
        res = Optim.optimize(
            λ -> mse_and_beta(λ, training, test_input, test_output, medium)[1],
            Float32[average_item_rating(training), 0, 0, -1, 0, 0],
            Optim.NewtonTrustRegion(),
            autodiff = :forward,
            Optim.Options(
                show_trace = true,
                extended_trace = true,
                g_tol = Float64(sqrt(eps(Float32))),
                time_limit = 3600,
            ),
        )
        λ = Optim.minimizer(res)
    else
        λ = read_params("$medium/Baseline/$version/training/rating")["λ"]
    end

    mse, β = mse_and_beta(λ, training, test_input, test_output, medium)
    @info "The optimal λ, mse is $λ, $mse"
    _, a = train_model(λ, training, medium)
    a_counts = get_counts(training, :itemid)
    write_params(
        Dict("λ" => λ, "β" => β, "a" => a, "a_counts" => a_counts),
        "$medium/Baseline/$version/$dataset/rating",
    )
end;

In [ ]:
@time build_model(medium, dataset, version; max_output_days = 7, max_output_items = 5);